In [1]:
import pandas as pd
from surprise import accuracy, Dataset, SVD, SVDpp, NMF
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate


ModuleNotFoundError: No module named 'surprise'

In [2]:
data = Dataset.load_builtin(name='ml-100k', prompt=True)


Dataset ml-100k could not be found. Do you want to download it? [Y/n] Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\User/.surprise_data/ml-100k


In [3]:
benchmark = []
algorithms = [SVD(), SVDpp(), NMF()]

for algorithm in algorithms:
    results = cross_validate(algorithm, data, measures=[
                             'RMSE'], cv=5, verbose=False)
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = pd.concat([pd.Series([str(algorithm).split(
        ' ')[0].split('.')[-1]], index=['Algorithm']), tmp])
    benchmark.append(tmp)


In [4]:
surprise_results = pd.DataFrame(benchmark).set_index(
    'Algorithm').sort_values('test_rmse')


In [5]:
surprise_results


,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.919538,16.109844,3.380188
SVD,0.935677,0.977418,0.153744
NMF,0.963889,1.389467,0.117471


In [6]:
trainset, testset = train_test_split(data, test_size=0.25)
algo = SVDpp()
algo.fit(trainset)
predictions = algo.test(testset)

accuracy.rmse(predictions)


RMSE: 0.9209


0.9209044899882067

Додаткове завдання


In [10]:
import numpy as np
from scipy import optimize
from scipy.io import loadmat

In [11]:
movie_ids = 'movie_ids.txt'
movies_mat = 'movies.mat'

In [12]:
def normalizeRatings(Y, R):
    m, n = Y.shape
    Ymean = np.zeros(m)
    Ynorm = np.zeros(Y.shape)

    for i in range(m):
        idx = R[i, :] == 1
        Ymean[i] = np.mean(Y[i, idx])
        Ynorm[i, idx] = Y[i, idx] - Ymean[i]

    return Ynorm, Ymean


def loadMovieList():
    # Read the fixed movieulary list
    with open(movie_ids,  encoding='ISO-8859-1') as fid:
        movies = fid.readlines()

    movieNames = []
    for movie in movies:
        parts = movie.split()
        movieNames.append(' '.join(parts[1:]).strip())
    return movieNames


def computeNumericalGradient(J, theta, e=1e-4):
    numgrad = np.zeros(theta.shape)
    perturb = np.diag(e * np.ones(theta.shape))
    for i in range(theta.size):
        loss1, _ = J(theta - perturb[:, i])
        loss2, _ = J(theta + perturb[:, i])
        numgrad[i] = (loss2 - loss1)/(2*e)
    return numgrad


def checkCostFunction(cofiCostFunc, lambda_=0.):
    # Create small problem
    X_t = np.random.rand(4, 3)
    Theta_t = np.random.rand(5, 3)

    # Zap out most entries
    Y = np.dot(X_t, Theta_t.T)
    Y[np.random.rand(*Y.shape) > 0.5] = 0
    R = np.zeros(Y.shape)
    R[Y != 0] = 1

    # Run Gradient Checking
    X = np.random.randn(*X_t.shape)
    Theta = np.random.randn(*Theta_t.shape)
    num_movies, num_users = Y.shape
    num_features = Theta_t.shape[1]

    params = np.concatenate([X.ravel(), Theta.ravel()])
    numgrad = computeNumericalGradient(
        lambda x: cofiCostFunc(x, Y, R, num_users, num_movies, num_features, lambda_), params)

    cost, grad = cofiCostFunc(params, Y, R, num_users,num_movies, num_features, lambda_)

    print(np.stack([numgrad, grad], axis=1))
    print('\nThe above two columns you get should be very similar.'
          '(Left-Your Numerical Gradient, Right-Analytical Gradient)')

    diff = np.linalg.norm(numgrad-grad)/np.linalg.norm(numgrad+grad)
    print('If your cost function implementation is correct, then '
          'the relative difference will be small (less than 1e-9).')
    print('\nRelative Difference: %g' % diff)

In [13]:
names = loadMovieList()
len(names)

1682

In [14]:

data = loadmat(movies_mat)
Y, R = data['Y'], data['R']

print('Average rating for movie 1601 (',names[1400] ,'): %f / 5' %
      np.mean(Y[180, R[1400, :]]))

Average rating for movie 1601 ( M. Butterfly (1993) ): 4.904560 / 5


In [15]:
def cofiCostFunc(params, Y, R, num_users, num_movies,
                      num_features, lambda_=0.0):
    # Unfold the U and W matrices from params
    X = params[:num_movies*num_features].reshape(num_movies, num_features)
    Theta = params[num_movies*num_features:].reshape(num_users, num_features)

    # You need to return the following values correctly
    J = 0
    X_grad = np.zeros(X.shape)
    Theta_grad = np.zeros(Theta.shape)

    J = (1 / 2) * np.sum(np.square((X.dot(Theta.T) - Y) * R)) + (lambda_ / 2) * np.sum(np.square(X)) + \
                                                                (lambda_ / 2) * np.sum(np.square(Theta))
    
    for i in range(R.shape[0]):
        
        idx = np.where(R[i, :] == 1)[0]
        Theta_temp = Theta[idx, :]
        Y_temp = Y[i, idx]
        X_grad[i, :] = np.dot(np.dot(X[i, :], Theta_temp.T) - Y_temp, Theta_temp) + lambda_ * X[i, :]
        
    for j in range(R.shape[1]):
        
        idx = np.where(R[:, j] == 1)[0]
        X_temp = X[idx, :]
        Y_temp = Y[idx, j]
        Theta_grad[j, :] = np.dot(np.dot(X_temp, Theta[j, :]) - Y_temp, X_temp) + lambda_ * Theta[j, :]
    
    grad = np.concatenate([X_grad.ravel(), Theta_grad.ravel()])
    return J, grad

In [16]:
checkCostFunction(cofiCostFunc, 1.5)

[[ 9.63355895e-01  9.63355895e-01]
 [ 3.82458375e-01  3.82458375e-01]
 [ 1.89468096e+00  1.89468096e+00]
 [ 1.99989831e+00  1.99989831e+00]
 [-3.52601200e+00 -3.52601200e+00]
 [-6.25417787e+00 -6.25417787e+00]
 [ 2.04001028e-01  2.04001028e-01]
 [-3.25317988e+00 -3.25317988e+00]
 [-3.14062405e+00 -3.14062405e+00]
 [-4.02625501e+00 -4.02625501e+00]
 [-4.60809805e+00 -4.60809805e+00]
 [ 2.29418475e-01  2.29418475e-01]
 [-3.25295539e+00 -3.25295539e+00]
 [-6.19889106e-01 -6.19889106e-01]
 [ 1.46498118e+00  1.46498118e+00]
 [-1.17467230e+00 -1.17467230e+00]
 [-1.91882696e+00 -1.91882696e+00]
 [-8.68432339e-01 -8.68432339e-01]
 [-2.41858048e+00 -2.41858048e+00]
 [ 4.62270383e+00  4.62270383e+00]
 [ 7.43647099e+00  7.43647099e+00]
 [-4.97316586e-01 -4.97316586e-01]
 [-4.92409967e-03 -4.92409966e-03]
 [ 2.28596628e+00  2.28596628e+00]
 [ 4.28078851e+00  4.28078851e+00]
 [ 3.23882654e+00  3.23882654e+00]
 [-2.41669293e+00 -2.41669293e+00]]

The above two columns you get should be very similar.

In [17]:
movieList = loadMovieList()

#  Before we will train the collaborative filtering model, we will first
#  add ratings that correspond to a new user that we just observed. This
#  part of the code will also allow you to put in your own ratings for the
#  movies in our dataset!
n_m = len(movieList)
#  Initialize my ratings
my_ratings = np.zeros(n_m)
# We have selected a few movies we liked / did not like and the ratings we
# gave are as follows:
my_ratings[0] = 4
my_ratings[97] = 2
my_ratings[6] = 2
my_ratings[11]= 5
my_ratings[53] = 4
my_ratings[63] = 5
my_ratings[364] = 2
my_ratings[68] = 5
my_ratings[182] = 4
my_ratings[225] = 5
my_ratings[354] = 1
my_ratings[22] = 5
my_ratings[124] = 5
my_ratings[54] = 2

In [18]:
#  Add our own ratings to the data matrix
Y = np.hstack([my_ratings[:, None], Y])
R = np.hstack([(my_ratings > 0)[:, None], R])

#  Normalize Ratings
Ynorm, Ymean = normalizeRatings(Y, R)

#  Useful Values
num_movies, num_users = Y.shape
num_features = 7

# Set Initial Parameters (Theta, X)
X = np.random.randn(num_movies, num_features)
Theta = np.random.randn(num_users, num_features)

initial_parameters = np.concatenate([X.ravel(), Theta.ravel()])

# Set options for scipy.optimize.minimize
options = {'maxiter': 100}

# Set Regularization
lambda_ = 10
res = optimize.minimize(lambda x: cofiCostFunc(x, Ynorm, R, num_users,
                                               num_movies, num_features, lambda_),
                        initial_parameters,
                        method='TNC',
                        jac=True,
                        options=options)
theta = res.x

# Unfold the returned theta back into U and W
X = theta[:num_movies*num_features].reshape(num_movies, num_features)
Theta = theta[num_movies*num_features:].reshape(num_users, num_features)

C:\Users\User\AppData\Local\Temp\ipykernel_3760\1892729221.py:23: DeprecationWarning: 'maxiter' has been deprecated in favor of 'maxfun' and will be removed in SciPy 1.11.0.
  res = optimize.minimize(lambda x: cofiCostFunc(x, Ynorm, R, num_users,


In [19]:
# Make recommendations by computing the predictions matrix
p = np.dot(X, Theta.T)
my_predictions = p[:, 0] + Ymean

movieList = loadMovieList()

ix = np.argsort(my_predictions)[::-1]

print('Top recommendations for you:')
print('----------------------------')
for i in range(15):
    j = ix[i]
    print('Predicting rating %.1f for movie %s' % (my_predictions[j], movieList[j]))

print('\nOriginal ratings provided:')
print('--------------------------')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print('Rated %d for %s' % (my_ratings[i], movieList[i]))

Top recommendations for you:
----------------------------
Predicting rating 5.0 for movie Santa with Muscles (1996)
Predicting rating 5.0 for movie Someone Else's America (1995)
Predicting rating 5.0 for movie Great Day in Harlem, A (1994)
Predicting rating 5.0 for movie They Made Me a Criminal (1939)
Predicting rating 5.0 for movie Marlene Dietrich: Shadow and Light (1996)
Predicting rating 5.0 for movie Entertaining Angels: The Dorothy Day Story (1996)
Predicting rating 5.0 for movie Saint of Fort Washington, The (1993)
Predicting rating 5.0 for movie Prefontaine (1997)
Predicting rating 5.0 for movie Star Kid (1997)
Predicting rating 5.0 for movie Aiqing wansui (1994)
Predicting rating 4.6 for movie Pather Panchali (1955)
Predicting rating 4.6 for movie Shawshank Redemption, The (1994)
Predicting rating 4.6 for movie Schindler's List (1993)
Predicting rating 4.5 for movie Star Wars (1977)
Predicting rating 4.5 for movie It's a Wonderful Life (1946)

Original ratings provided:
------